### **JSON 출력 파서(JsonOutputParser)**

사용자가 원하는 JSON 스키마를 지정, 그 스키마에 맞게 LLM에서 데이터를 조회하여 결과 도출 

테스트 해보니 JsonOutputParser, PydanticOutputParser는 거의 같은거같음 Pydantic은 dict()을 해주면 Json과 같음

In [ ]:
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq

In [ ]:
# 출력 구조 정의
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간략한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상")

### **ChatPromptTemplate**

In [ ]:
# 출력 파서 정의
parser = JsonOutputParser(pydantic_object=Topic)

# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 사용자의 질문에 대답하는 어시스턴트 입니다 간결하게 답변해주세요'),
    ('user', '#Format: {format_instructions}\n\n{question}')]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

llm = ChatGroq(model = 'gemma2-9b-it')

chain = prompt | llm | parser

chain.invoke(question)

### **PromptTemplate.from_template**

In [ ]:
# 출력 파서 정의
parser = JsonOutputParser(pydantic_object=Topic)

# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

prompt = PromptTemplate.from_template(
    """
    당신은 사용자에게 친절하게 답변해주는 AI입니다. 질문에 간결하게 답변해주세요'    
    
    #Format: {format_instructions}
    
    Question : {question}
    """
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | llm | parser

chain.invoke({'question': question})

### **PromptTemplate**

In [ ]:
prompt = PromptTemplate(
    template = '사용자의 질문에 친절하게 답변해주는 어시스턴트 입니다. 간결하게 답변해주세요\n\n#Format: {format_instructions}\n\n{question}',
    input_variables = ['question'],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

chain = prompt | llm | parser

chain.invoke('지구 온난화의 심각성 대해 알려주세요.')

### **Pydantic 없이 사용**

In [ ]:

parser = JsonOutputParser()

question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

prompt = PromptTemplate.from_template(
    """
    당신은 사용자에게 친절하게 답변해주는 AI입니다. 질문에 간결하게 답변해주세요'    
    
    #Format: {format_instructions}
    
    Question : {question}
    """
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

llm = ChatGroq(model = 'gemma2-9b-it')

chain = prompt | llm | parser

chain.invoke(question)

{'description': '지구 온난화는 지구 평균 기온이 오랜 시간 동안 상승하는 현상을 말합니다. 주로 인간 활동으로 인해 발생하는 온실가스 배출이 원인이며, 해수면 상승, 극심한 기후 현상, 생물 다양성 감소 등 심각한 영향을 미치고 있습니다.',
 'hashtags': ['#지구온난화', '#온실가스', '#기후변화', '#환경문제', '#지속가능성']}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | llm | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)

{'description': '지구 온난화는 지구의 평균 기온이 장기간 증가하는 현상으로, 대기 중 온실가스 증가로 인해 발생합니다. 이는 해빙, 해수면 상승, 기후 변화 등 심각한 환경 문제를 야기합니다.', 'hashtags': ['#지구온난화', '#온실가스', '#기후변화', '#환경문제', '#해빙', '#해수면상승']}
